## Question 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
import time
from imageio import imread, imwrite
from scipy import ndimage

In [6]:
def read(file_name, image_type):
    img = cv2.imread(file_name)
    if image_type == "RGB":
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif image_type == "HSV":
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)


def task_one(img, s, k, alpha):
    kernel, blurred, mask, img = sharpen_image_one(img, s, k, alpha)
    mask = mask + 128

    mat = np.zeros((101, 101))
    mat[50, 50] = 255
    kernel = cv2.filter2D(mat, ddepth=-1, kernel=kernel)
    kernel = cv2.normalize(kernel, 0, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite("res01.jpg", img)


def sharpen_image_one(img, s, k, alpha):
    img = img.astype('float64')
    kernel = get_gaussian_kernel(s, k)
    blurred = cv2.filter2D(img, ddepth=-1, kernel=kernel)
    mask_1 = img - blurred
    mask = alpha * (img - blurred)
    img = img + alpha * mask
    img[img < 0] = 0
    img[img > 255] = 255
    return kernel, blurred, mask_1, img


def get_gaussian_kernel(s, k):
    n = 2 * k + 1
    return np.array([[gaussian(k, k, i, j, s) / (2 * pi * s ** 2) for j in range(n)] for i in range(n)])


def gaussian(m, n, i, j, sigma):
    return np.exp(-((i - m) ** 2 + (j - n) ** 2) / (2 * sigma ** 2))


def task_two(img, s, k):
    kernel, mask, img = sharpen_image_two(img, s, k)
    mask = mask + 128

    mat = np.zeros((101, 101))
    mat[50, 50] = 255
    kernel = cv2.filter2D(mat, ddepth=-1, kernel=kernel)
    kernel = cv2.normalize(kernel, 0, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite("res02.jpg", img)


def sharpen_image_two(img, s, k):
    img = img.astype('float64')
    kernel = get_laplacian(s, 9)
    mask_1 = cv2.filter2D(img, ddepth=-1, kernel=kernel)
    img = img - k * mask_1
    img[img < 0] = 0
    img[img > 255] = 255
    return kernel, mask_1, img


def get_laplacian(sigma, n):
    kernel = np.array([[laplacian(sigma, (i - (n - 1) / 2), (j - (n - 1) / 2)) for j in range(n)] for i in range(n)])
    return kernel


def laplacian(sigma, x, y):
    laplace = (((x ** 2 + y ** 2) - (2 * sigma ** 2)) * np.exp(-(x ** 2 + y ** 2) / (2 * sigma ** 2))) / (
            np.pi * sigma ** 4)
    return laplace


def get_kernel(rows, cols, s, kernel_type):
    m, n = np.floor(rows / 2).astype(int), np.floor(cols / 2).astype(int)
    kernel = np.array([[gaussian(m, n, i, j, s) for j in range(cols)] for i in range(rows)])
    if kernel_type == "High_pass":
        return np.ones(kernel.shape) - kernel
    else:
        return kernel


def task_three(img, sigma, alpha):
    img_f, high_pass, mask, res_image = sharpen_image_three(img, sigma, alpha)
    img_f = np.log(np.abs(img_f))
    high_pass = np.abs(high_pass)
    mask = np.abs(mask)
    img_f = cv2.normalize(img_f, 0, 0, 255, cv2.NORM_MINMAX)
    high_pass = cv2.normalize(high_pass, 0, 0, 255, cv2.NORM_MINMAX)
    mask = cv2.normalize(mask, 0, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite("res03.jpg", res_image)


def sharpen_image_three(img, sigma, alpha):
    img = img.astype('float64')
    b, g, r = cv2.split(img)
    h, w, _ = img.shape
    kernel = get_kernel(h, w, sigma, "High_pass")
    f_r, mask_r, res_r = apply_fourier(r, kernel, alpha)
    f_g, mask_g, res_g = apply_fourier(g, kernel, alpha)
    f_b, mask_b, res_b = apply_fourier(b, kernel, alpha)
    res_f = np.dstack((f_b, f_g, f_r))
    res_mask = np.dstack((mask_b, mask_g, mask_r))
    res_image = np.dstack((res_b, res_g, res_r))
    img[img < 0] = 0
    img[img > 255] = 255
    return res_f, kernel, res_mask, res_image


def apply_fourier(channel, kernel, alpha):
    img_f = np.fft.fft2(channel)
    img_shifted = np.fft.fftshift(img_f)
    mask = 1 + alpha * kernel
    res_image = mask * img_shifted
    res_image = np.fft.ifftshift(res_image)
    res_image = np.fft.ifft2(res_image)
    res_image = np.real(res_image)
    return img_shifted, mask, res_image


def task_four(img, alpha):
    img = img.astype('float64')
    b, g, r = cv2.split(img)
    h, w = r.shape
    h2, w2 = h // 2, w // 2
    indexes = np.array([[np.abs(i - h2) ** 2 + np.abs(j - w2) ** 2 for j in range(w)] for i in range(h)])
    r_mask, r_f, r = apply(r, indexes, alpha)
    g_mask, g_f, g = apply(g, indexes, alpha)
    b_mask, b_f, b = apply(b, indexes, alpha)
    res_mask = np.dstack((b_mask, g_mask, r_mask))
    res_mask = np.abs(res_mask)
    res_f = np.dstack((b_f, g_f, r_f))
    res_f = np.abs(res_f)
    res_f = res_f / np.max(res_f) * 255
    res_f = 128 + res_f
    res_image = np.dstack((b, g, r))
    cv2.imwrite("res04.jpg", res_image)


def apply(channel, indexes, alpha):
    img_f = np.fft.fft2(channel)
    img_shifted = np.fft.fftshift(img_f)
    mask = (4 * pi ** 2) * indexes * img_shifted
    img_back = np.fft.ifftshift(mask)
    img_back = np.fft.ifft2(img_back)
    res_image = channel + alpha * img_back
    res_image = np.abs(res_image)
    res_image[res_image < 0] = 0
    res_image[res_image > 255] = 255
    return mask, img_back, res_image


In [12]:
image = cv2.imread("image1.jpeg")

start_time = time.time()
task_one(image, 1, 2, 2)
print("execution time:", time.time() - start_time)

start_time = time.time()
task_two(image, 1, 2)
print("execution time:", time.time() - start_time)

start_time = time.time()
task_three(image, 300, 20)
print("execution time:", time.time() - start_time)

start_time = time.time()
task_four(image, 0.000003)
print("execution time:", time.time() - start_time)

execution time: 0.04521536827087402
execution time: 0.04064369201660156
execution time: 0.8612821102142334
execution time: 0.6465568542480469


## Question 2

<div dir="rtl">

همانطور که در تصاویر خروجی مشخص است همگی بسته به پارامتر‌های داده شده تصویر خروجی را sharp میکنند اما در تصاویری که از فوریه برای sharp کردن استفاده شده چون تصویر اصلی smooth نبوده و به دلیل ذخیره سازی فشرده شده است و مربع‌هایی در تصویر دیده می‌شود، در تبدیل فوریه این مربعات تشدید شده و باعث ایجاد نویز‌هایی در تصویر شده است. همچنین به لحاظ زمان‌بندی دو روش اول که از تبدیل فوریه استفاده نمی‌کنند سریع‌تر هستند.
</div>

In [42]:
img = imread('image2.png')

### Sobel filter

In [43]:
kernel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
kernel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

sobel_x = ndimage.convolve(img, kernel_x)
sobel_y = ndimage.convolve(img, kernel_y)
sobel = np.sqrt(sobel_x**2+sobel_y**2)
sobel = sobel - np.min(sobel)
sobel = sobel / np.max(sobel) * 255

imwrite('q5_res01.png', np.uint8(sobel))

### Gaussian filter

In [46]:
def gaussian_kernel(n, sigma):
    return [[1/(2*np.pi*sigma**2) * np.exp(-((i-n//2)**2+(j-n//2)**2)/(2*sigma**2)) for j in range(n)] for i in range(n)]

kernel = gaussian_kernel(3,1)
gaussian = np.uint8(ndimage.convolve(img, kernel))

imwrite('q5_res02.png', gaussian)

### Laplacian filter

In [45]:
kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])
laplacian = ndimage.convolve(img, kernel)

imwrite('q5_res03.png', laplacian)

### Frecuency domain

In [47]:
freq_img = np.fft.fftshift(np.fft.fft2(img))
imwrite('q5_res04.png', np.uint8(20*np.log10(np.abs(freq_img))))

### Gaussian

In [51]:
def gaussian_filter(img, sigma):
    x, y = img.shape
    i = np.linspace(0, x, x)
    j = np.linspace(0, y, y)
    filt = np.outer(np.exp(-((i-x//2)**2)/(2*sigma**2)), np.exp(-((j-y//2)**2)/(2*sigma**2)))
    return filt/np.max(filt)


lp_img_f = freq_img*gaussian_filter(freq_img, 100)
imwrite('q5_res05.png', np.uint8(20*np.log10(np.abs(lp_img_f)+1e-3)))

hp_img_f = freq_img*(1-gaussian_filter(freq_img, 100))
imwrite('q5_res06.png', np.uint8(20*np.log10(np.abs(hp_img_f)+1e-3)))

In [52]:
lp_img = np.abs(np.fft.ifft2(np.fft.ifftshift(lp_img_f)))
hp_img = np.abs(np.fft.ifft2(np.fft.ifftshift(hp_img_f)))

lp_img = (lp_img-np.min(lp_img))
lp_img = lp_img/(np.max(lp_img)-np.min(lp_img))*255
hp_img = (hp_img-np.min(hp_img))
hp_img = hp_img/(np.max(hp_img)-np.min(hp_img))*255

imwrite('q5_res07.png', np.uint8(lp_img))
imwrite('q5_res08.png', np.uint8(hp_img))

### Squared

In [54]:
def squared_filter(img, cutoff):
    x, y = img.shape
    filt = np.zeros(img.shape)
    filt[x//2-cutoff:x//2+cutoff, y//2-cutoff:y//2+cutoff] = 1
    return filt/np.max(filt)

lp_img_f = freq_img*squared_filter(freq_img, 50)
imwrite('q5_res09.png', np.uint8(20*np.log10(np.abs(lp_img_f)+1e-3)))

hp_img_f = freq_img*(1-squared_filter(freq_img, 50))
imwrite('q5_res10.png', np.uint8(20*np.log10(np.abs(hp_img_f)+1e-3)))

In [55]:
lp_img = np.abs(np.fft.ifft2(np.fft.ifftshift(lp_img_f)))
hp_img = np.abs(np.fft.ifft2(np.fft.ifftshift(hp_img_f)))

lp_img = (lp_img-np.min(lp_img))/(np.max(lp_img)-np.min(lp_img))*255
hp_img = (hp_img-np.min(hp_img))/(np.max(hp_img)-np.min(hp_img))*255

imwrite('q5_res11.png', np.uint8(lp_img))
imwrite('q5_res12.png', np.uint8(hp_img))

<div dir="rtl">

آ) در خروجی فیلتر گوسی، بخش های دور تصویر که متشکل از نوارهای با فرکانس بالاتر می باشند،
تصعیف شده اند.
در خروجی فیلتر لاپلاسین، لبه های تصویر  sharp تر شده است همچنین پهنای هر کدام از نوار های دایروی کمتر شده است نکته
جالب این است که وسط تصویر به جای دایره سفید در تصویر اصلی، دایره کوچکتری با رنگ سیاه به وجود آمده است.
دلیل این اتفاق این است که فیلتر لاپلاسین مشتق دوم را را نمایش می دهد و به دلیل اینکه در وسط تصویر و در بازه مشخصی میزان
روشنایی تقریبا ثابت است، مشتقات اول و دوم هردو صفر بوده و به همین دلیل دایره تیره رنگ در وسط صفحه تشکیل شده است.
فیلتر سوبل نیز نقش تشخیص لبه در تصویر را انجام می دهد و به همین دلیل در نقاط مرزی لبه های تصویر، خروجی فیلتر سوبل
روشنایی بیشتر و در نقاط وسط هر کدام از نوار های تصویر اصلی، خروجی فیلتر سوبل تیره تر می باشد که قابل انتظار است.

ب)
برای اثبات گوسی بودن تبدیل فوریه فیلتر گوسی از آنجا که فیلتر گوسی جدایی پذیر است، کافیست برای یک فیلتر گوسی یک بعدی این موضوع را
اثبات کنیم و در این صورت برای فیلتر گوسی دو بعدی نیز اثبات می شود :
</div>

$f(u) = \int \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{x^2}{2\sigma^2}} e^{-2j\pi ux} dx$

$\frac{df(u)}{du} = -\frac{j\sqrt{2\pi}}{\sigma} \int e^{-2j\pi ux} x e^{-\frac{x^2}{2\sigma^2}} dx = -4\pi^2\sigma u f(u)$

$\frac{df(u)}{f(u)} = -2\pi^2\sigma^2 du^2 → ln(\frac{f(u)}{f(0)}) = -2\pi^2\sigma^2u^2 → f(u) = f(0)e^{-2\pi^2\sigma^2u^2}, f(0) = \int \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{x^2}{2\sigma^2}} dx = \sqrt{2\pi}\sigma → \sqrt{2\pi} \sigma e^{2\pi^2\sigma^2u^2}$

و در نهایت برای حالت دو بعدی خواهیم داشت:

$f(u, v) = 2\pi\sigma^2 e^{-2\pi^2\sigma^2(u^2 + v^2)}$

<div dir="rtl">

 تصویر به دست آمده ناشی از اعمال فیلتر گوسی تفاوتی با تصویر ناشی از
کانوالو کردن فیلتر در حوزه زمان که در قسمت قبل به دست آوردیم ندارد. در رابطه با متمم فیلتر گوسی تصویر خروجی در نقاط
وسط که فرکانس بالاتری دارد تیره تر شده است و همچنین نوارهای دور صفحه واضح تر و sharp تر شده اند.
در مقابل، با اعمال فیلتر مربعی مشاهده می کنیم که باز هم بخش فرکانس بالا حذف شده اما نکته قابل توجه این است که شکل کلی تصویر هم به
شکل مربعی در می آید. و باز هم با اعمال متمم مشاهده می شود که بخش های وسط تصویر که دارای فرکانس پایین هستند کاملا از بین رفته اما به دلیل
مربعی بودن خود فیلتر بخش های فرکانس بالا نیز دچار تغییراتی می شوند.
دلیل به وجود آمدن مربع های کوچک روی تصویر این است که وقتی در حوزه‌ی فرکانس این فیلتر مربعی را در تصویر اعمال می‌کنیم، چون این فیلتر معادل ضرب دو تابع sinc در دو راستای x و y است و چون در حوزه‌ی زمان این فیلتر اعمال می‌شود در برخی نقاط برآیند ضرب این دو تابع در دره و قله ها مثادیر زیاد و کم می‌شود و این باعث به وجود آمدن مربع‌های کوچک در تصاویر می‌شود. مثلا در بخشی از تصویر که کاملا سفید است با جابه‌جا شدن این فیلتر قله‌ها و دره‌ها به صورت متناوب ضرایب زیاد و کم میگیرند.
</div>